# Loading up the Data
The data will be loaded from the following two files

- data/train_events.csv
- data/train_series.parquet

into two pandas dataframe `events` and `series`

In [1]:
import pandas as pd

In [2]:
events = pd.read_csv('data/train_events.csv')
events.head()

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [3]:
series = pd.read_parquet("data/train_series.parquet", engine='fastparquet')

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0.0,2018-08-14T15:30:00-0400,2.6367,0.0217
1,038441c925bb,1.0,2018-08-14T15:30:05-0400,2.6368,0.0215
2,038441c925bb,2.0,2018-08-14T15:30:10-0400,2.6370,0.0216
3,038441c925bb,3.0,2018-08-14T15:30:15-0400,2.6368,0.0213
4,038441c925bb,4.0,2018-08-14T15:30:20-0400,2.6368,0.0215


In [6]:
series.head()

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0.0,2018-08-14T15:30:00-0400,2.6367,0.0217
1,038441c925bb,1.0,2018-08-14T15:30:05-0400,2.6368,0.0215
2,038441c925bb,2.0,2018-08-14T15:30:10-0400,2.6370,0.0216
3,038441c925bb,3.0,2018-08-14T15:30:15-0400,2.6368,0.0213
4,038441c925bb,4.0,2018-08-14T15:30:20-0400,2.6368,0.0215


In [4]:
len(series), len(events)

(127946340, 14508)